## Dependencies

In [1]:
import discord
from discord.ext import commands

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams

from datetime import datetime, timedelta  

import io
import os
import math
import random

from imdb import IMDb


# Load necessary tokens and IDs from .env file (not included in repository)
from dotenv import load_dotenv
_ = load_dotenv()

## Bot Setup

In [30]:
# Global running list of winning movie titles for the current week
titles = []

# Global list of movies returned from IMDB
movies = []

# List of past winners, stored in clumsy-movie-winners.csv
winners = pd.read_csv('clumsy-movie-winners.csv', dtype = {'Title':  np.str, 'ID': np.str})

# Account/channel specific information stored as environmental variable
token = os.environ["DISCORD_TOKEN"]
channel_ID = int(os.environ["DISCORD_CHANNEL"])
terminal_ID = int(os.environ["DISCORD_TERMINAL"])
test_ID = int(os.environ["DISCORD_TEST"])

In [21]:
# Top 1000 B-movies on IMDB by popularity
bmovies = []
for i in range(1,21):
    bmovies.extend(IMDb().get_keyword(keyword='b-movie',page=i))

In [22]:
def lastSaturday():
    
    """Votes are pulled from the most recent Saturday after 10 PM/22:00 EST (UTC-4:00)"""
    
    if(datetime.today().weekday() == 5):
        lastSat = datetime.today() - timedelta(days = 7)
        return lastSat.replace(day=lastSat.day + 1, hour=2, minute=0, second=0)
    else:
        idx = (datetime.today().weekday() + 1) % 7
        lastSat = datetime.today() - timedelta(7+idx-6)
        return lastSat.replace(day = lastSat.day + 1, hour=2, minute=0, second = 0)  


async def isTerminal(ctx):
    
    """ Custom check used in all commands to limit bot commands to skynet terminal OR clumsy testing server """
        
    if(ctx.command.name == 'rollover'):
        return True
    
    global terminal_ID
    return ctx.channel.id == terminal_ID or ctx.channel.id == test_ID


In [29]:
# All commands for bot will be prefixed with a period (e.g. '.help')
client = commands.Bot(command_prefix = '.')
client.add_check(isTerminal)

# Message bot will print to console when it is connected and ready to receive commands
@client.event 
async def on_ready():
    print('Bot is ready. Use kill command to log bot out.')

## Wheel Commands

In [24]:
client.remove_cog('1: Voting')

class Voting(commands.Cog, name='1: Voting'):
    """Commands to read and summarize movie nominations and voting"""

    def __init__(self, bot):
        self.bot = bot
        
        
    @commands.command(brief='Tally votes', 
                    description='Generates a bar chart of votes for all movies that received at least one reaction since last Saturday at 10:00 (UTC time)')
    async def tally(self, ctx):
        await ctx.send("Tabulating votes...")

        channel = client.get_channel(channel_ID)

        # Add movies and number of reactions (i.e. votes) and sort in descending order

        votes = []
        number_of_votes = 0

        async for message in channel.history(after=lastSaturday()):
            if len(message.reactions) > 0 and message.content not in titles:
                
                number_of_votes = 0
                
                for reaction in message.reactions:
                    number_of_votes += reaction.count
                
                votes.append((message.content, number_of_votes))

        votes = pd.DataFrame.from_records(np.array(votes), columns = ['Movie', 'Number of Votes'])
        votes["Number of Votes"] = pd.to_numeric(votes["Number of Votes"])
        votes.sort_values(by = "Number of Votes", ascending = False, inplace = True)

        # Create horizontal bar chart of movie rankings

        rcParams.update({'figure.autolayout': True})
        rcParams.update({'figure.figsize': [16,9]})

        fig, ax = plt.subplots()

        movies = votes["Movie"]
        movies_range = np.arange(len(movies))
        ranking = votes["Number of Votes"]

        ax.barh(movies_range, ranking, align='center')
        ax.set_yticks(movies_range)
        ax.set_yticklabels(movies)
        ax.invert_yaxis()  # labels read top-to-bottom
        ax.set_xlabel('Number of Votes')
        ax.set_title('Clumsy Movie Ranking ' + "(as of " + datetime.now().strftime("%m/%d/%Y, %H:%M:%S") + ")")

        # Save figure locally and then embed into message 

        fig.savefig('discord-images/graph.png')

        with open('discord-images/graph.png', 'rb') as f:
            file = io.BytesIO(f.read())    

        image = discord.File(file, filename='graph.png')
        embed = discord.Embed(title = "Votes as of " + datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
        embed.set_image(url=f'attachment://graph.png')

        await ctx.send(file=image, embed=embed)
        
        
    @commands.command(brief='Prepare votes for the wheel', description='Generates a list for all movies that received at least one reaction since last Saturday at 9:30 (UTC time). Movie titles are duplicated according to number of votes.')
    async def wheel(self, ctx):

        channel = client.get_channel(channel_ID)
        
        # Create a text list of all movie titles, copied according to number of votes

        wheel_list = ""
        number_of_votes = 0

        await ctx.send("Wheel List:\n")        
        
        async for message in channel.history(after=lastSaturday()):
            if len(message.reactions) > 0 and message.content not in titles:
                
                number_of_votes = 0
                
                for reaction in message.reactions:
                    number_of_votes += reaction.count                
                
                if(len(wheel_list + (message.content + "\n") * number_of_votes) >= 2000):
                    await ctx.send(wheel_list)
                    wheel_list = ""
                
                wheel_list = wheel_list + (message.content + "\n") * number_of_votes    
                

        await ctx.send(wheel_list) 
        

    @commands.command(brief='Added winning movie to temporary winner list', description='Add winning movie for the current week to a temporary list of winners. Run prior to rollover function.')
    async def winner(self, ctx, *, title: str):
        
        global titles
        titles.append(title)
        
        await ctx.send("Added winner: " + title)

        
    @commands.command(brief='Added winning movie to permanent winning list', description='Add winning movie for the current week to a permanent list of winners. Use index from most recent IMDB search to store title and IMDB ID.')
    async def winner2(self, ctx, index: int):

        try:
            global movies
            global winners
            index = int(index) - 1    
            movieID = movies[index].movieID
            movie = IMDb().get_movie(movieID)
        except IndexError:
            await ctx.send("Please run .imdb command first to store list of movies")
            return   

        
        await ctx.send("Added to Permanent Movie List: " + movie['title'])
        
        winners = winners.append({'Title': movie['title'], 'ID': movieID}, ignore_index=True)
        winners.to_csv('clumsy-movie-winners.csv', index = False)
            
            
    @commands.command(brief='List winners', description='Print the list of winners to be excluded from .rollover command')
    async def winner_list(self, ctx):
        
        global titles
        await ctx.send(titles)
        
        
    @commands.command(brief='Clear winners', description='Clear the winners list used in the .rollover command')
    async def winner_clear(self, ctx):
        
        global titles
        titles = []
        
        
    @commands.command(brief='Display past winners', description='Display a list of past winners')
    async def winners(self, ctx):
        
        global winners
        
        results = "Clumsy Movie Past Showings:\n"
        
        for i in range(len(winners)):
            results += "[" + str(i+1) + "] " + winners.iloc[i]['Title'] + "\n"
        
        await ctx.send(results)          
        
        
    @commands.command(brief='Create a rollover list', description='Create a rollover list for the next week, with movies that have at least 1 vote. NOTE: Add winners to winner list first with winner command')
    async def rollover(self, ctx):

        await ctx.send("Next Week on the Wheel:")

        channel = client.get_channel(channel_ID) 

        async for message in channel.history(after=lastSaturday()):
            if len(message.reactions) > 0 and message.content not in titles:
                await ctx.send(message.content)  
                
                
client.add_cog(Voting(client))

## IMDB Commands

In [25]:
client.remove_cog('2: IMDB Queries')

class IMDB_Queries(commands.Cog, name='2: IMDB Queries'):
    """Query the IMDB movie database"""

    
    def __init__(self, bot):
        self.bot = bot

        
    @commands.command(brief = 'Run IMDB search for specified title', description = 'Returns the top 10 results from IMDB for using the specified title as the search query')
    async def imdb(self, ctx, *, title: str):
        ia = IMDb()
        global movies
        movies = ia.search_movie(title)
        
        await ctx.send("One moment please...")
        
        results = "Top 10 Search Results from IMDB:\n"


        for i in range(len(movies)):

            results += "[" + str(i+1) + "] " + movies[i]['long imdb title'] + "\n"

            if(i == 9):
                break
                
        await ctx.send(results)


    @commands.command(brief = 'Show IMDB summary for selected movie', description = 'After running .imdb command, use the .imdb_summary <index> command to display the IMDB summary for a selected movie. If the .imdb command has not been run previously, an error message will be produced.')
    async def imdb_summary(self, ctx, index):

        ia = IMDb()

        try:
            global movies
            index = int(index) - 1    
            movieID = movies[index].movieID
            movie = ia.get_movie(movieID)
        except IndexError:
            await ctx.send("Please run .imdb command first to store list of movies")
            return

        
        try:
            title = movie['long imdb title']
        except KeyError:
            title = "Unavailable"

        try:
            description = movie['plot'][0].split('::')[0]
        except KeyError:
            description = "Unavailable"

        try:
            score = str(movie['rating'])
        except KeyError:
            score = "N/A"

        try:
            runtime = str(movie['runtimes'][0]) + " minutes"
        except KeyError:
            runtime = 'N/A'

        embed = discord.Embed(title = title, 
                              description = description,
                             colour = discord.Colour.blue(),
                             url = "https://www.imdb.com/title/tt" + movieID)
        
        embed.add_field(name = 'IMDB Score', value = score, inline = True)
        embed.add_field(name = 'Runtime', value = runtime, inline = True)      

        try:
            embed.set_image(url=movie['full-size cover url'])
        except KeyError:
            pass

        await ctx.send(embed=embed)           

        
    @commands.command(brief='Trivia for past winner', description='Display all IMDB trivia for past winner')
    async def trivia(self, ctx, index:int):
        
        global winners
        
        results = "Trivia for: " + winners.iloc[index-1]['Title'] + "\n\n"
        
        for trivia in IMDb().get_movie_trivia(winners.iloc[index-1]['ID'])['data']['trivia'][:10]:
            
            if(len(results + trivia + "\n") >= 2000):
                await ctx.send(results.replace("(qv)",""))
                results = ""            
            
            results += trivia + "\n\n"
        
        await ctx.send(results.replace("(qv)","") + "\n\n")  
        
        
    @commands.command(brief = 'Select random B-movie from IMDB Top 1000', description = '')
    async def random(self, ctx):

        ia = IMDb()
        global bmovies
        
        try:
            while True:
                index = random.randint(0,len(bmovies)-1)  
                movieID = bmovies[index].movieID
                movie = IMDb().get_movie(movieID)
                exclude = False

                if(movie['kind'] == 'movie' and movie['year'] >= 1950):
                    genres = ['War', 'News', 'Film-Noir', 'History', 'Biography', 'Documentary']
                    for genre in genres:
                        if genre in movie['genres']:
                            exclude = True

                    if exclude == False:
                        break
        except:
            await ctx.send("Something went wrong")
            return

        
        try:
            title = movie['long imdb title']
        except KeyError:
            title = "Unavailable"

        try:
            description = movie['plot'][0].split('::')[0]
        except KeyError:
            description = "Unavailable"

        try:
            score = str(movie['rating'])
        except KeyError:
            score = "N/A"

        try:
            runtime = str(movie['runtimes'][0]) + " minutes"
        except KeyError:
            runtime = 'N/A'

        embed = discord.Embed(title = title, 
                              description = description,
                             colour = discord.Colour.blue(),
                             url = "https://www.imdb.com/title/tt" + movieID)
        
        embed.add_field(name = 'IMDB Score', value = score, inline = True)
        embed.add_field(name = 'Runtime', value = runtime, inline = True)      

        try:
            embed.set_image(url=movie['full-size cover url'])
        except KeyError:
            pass

        await ctx.send(embed=embed)            
        
        
client.add_cog(IMDB_Queries(client))

## Utility Commands

In [26]:
client.remove_cog('3: Utility')

class Utility(commands.Cog, name='3: Utility'):
    """Helper commands to facilitate testing or manage bot"""

    def __init__(self, bot):
        self.bot = bot
        
        
    @commands.command(brief='Force logout for bot', description='Forces the bot to logoff Discord. Convenience function to interrupt process from jupyter notebook')
    async def kill(self, ctx):
        await ctx.send("Thank you for using Clumsy Movie Bot. Goodbye.")

        # Log bot out of Discord
        await client.logout()

        # Clear internal cache of bot and prepare it to be reopened if necessary
        client.clear()
      
    
    
    # For testing/debugging purposes
    
    @commands.command(brief='Delete all messages', description='Removes last 1000 messages before current datetime (UTC)')
    async def purge(self, ctx):
        channel = client.get_channel(channel_ID)

        # Removes the last 1000 messages in channel
        await channel.purge(limit = 1000, before = datetime.utcnow() + timedelta(1))  
        

    @commands.command(brief='Print 5 sample movies', description='Prints 5 seperate messages with a movie name. Reactions should be added to movie title to register vote.')
    async def samples(self, ctx):

        # Create sample movie nominations with emoji reactions to simulate votes

        m1 = await ctx.send("Lair of the White Worm") 
        m2 = await ctx.send("Hausu") 
        m3 = await ctx.send("Hackers") 
        m4 = await ctx.send("Earth Girls are Easy") 
        m5 = await ctx.send("50 Shades Darker") 

        await m1.add_reaction('\U0001f44d')
        await m2.add_reaction('\U0001f44d')
        await m3.add_reaction('\U0001f44d')
        await m4.add_reaction('\U0001f44d')
        await m5.add_reaction('\U0001f44d')

        await m4.add_reaction('\U0001f600')
        await m5.add_reaction('\U0001f600')

        await m4.add_reaction('\U0001f603')
    
    
    
client.add_cog(Utility(client))

## Connect Bot to Discord Server

If bot has not connected since computer startup, connect to discord:

In [27]:
await client.login(token, bot = True)

Connect bot to receive commands. Process will continue to run until interrupted in discord with **.kill** command

In [28]:
await client.connect(reconnect = True)

Bot is ready. Use kill command to log bot out.
